In [571]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
import scipy.io as sio
import os 
os.environ['KERAS_BACKEND'] = 'plaidml.keras.backend'

In [572]:
GER_test = (np.array(sio.loadmat('./../GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('./../GER_train.mat')['GER_train'])).transpose()

In [573]:
df_train = pd.DataFrame(GER_train)
df_test = pd.DataFrame(GER_test)

Preprocessing

In [574]:
del df_train[1]
del df_test[1]

In [575]:
df_train = pd.DataFrame(np.array(df_train.values.tolist()))
df_test = pd.DataFrame(np.array(df_test.values.tolist()))

In [576]:
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.046318,0.141910,0.084182,0.383329,0.139563,4.503520,-1.408795,6.647475,0.617802,0.094052,0.082395,0.069777,0.023589,4.426887,-0.164270,2.083033,0.360066,0.321429,0.500000
1,0.0,0.073971,0.087852,0.071337,0.180243,0.054842,6.338891,-0.645344,2.393239,0.382198,0.047816,0.044660,0.071250,0.027742,3.443768,0.475093,2.737538,0.194479,0.678571,0.500000
2,0.0,0.037709,0.061516,0.027380,0.159906,0.060017,4.312509,-0.654140,3.289271,0.189828,0.071038,0.065286,0.057454,0.014160,5.029416,-0.134997,1.937705,0.206033,0.159091,0.168539
3,0.0,0.088211,0.083918,0.081087,0.092759,0.017558,8.095644,0.162758,1.868488,0.267116,0.100426,0.096071,0.095724,0.021325,5.226386,0.239747,1.932730,0.302636,0.272727,0.123596
4,1.0,0.063538,0.065820,0.062985,0.062872,0.018931,7.696024,-0.682489,2.553929,0.207056,0.053089,0.051664,0.046562,0.011208,5.010536,-0.032268,1.960073,0.163216,0.181818,0.134831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6976,1.0,0.075688,0.116231,0.079073,0.313965,0.100537,4.813671,-1.512756,6.660133,0.306430,0.123269,0.121593,0.182902,0.055990,4.209587,-0.565761,2.472689,0.322430,0.466667,0.478261
6977,0.0,0.258272,0.254036,0.220336,0.393153,0.119428,7.099124,0.605659,2.548129,0.693570,0.208532,0.193873,0.217791,0.054846,5.097935,0.273143,1.997188,0.531131,0.533333,0.521739
6978,1.0,0.106475,0.137536,0.064309,0.302599,0.120021,5.708663,-1.248314,5.236364,0.440847,0.122621,0.115695,0.073606,0.025475,4.181087,-0.464833,2.518841,0.371926,0.274510,0.389610
6979,0.0,0.041224,0.043721,0.041356,0.066269,0.015119,9.613998,0.202926,1.489863,0.146142,0.027363,0.025981,0.026985,0.006103,5.217626,-0.239576,1.902420,0.085627,0.117647,0.129870


In [577]:
df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.057977,0.126516,0.088432,0.318507,0.104176,5.650807,-1.122731,4.591241,0.640278,0.077235,0.068464,0.071681,0.023683,4.413345,-0.119818,2.117172,0.345433,0.379310,0.571429
1,0.0,0.064399,0.071318,0.065064,0.085757,0.030750,6.950576,-0.967590,3.194839,0.359722,0.050524,0.048262,0.047164,0.016054,4.195770,0.329532,2.300845,0.243610,0.620690,0.428571
2,0.0,0.118056,0.113939,0.112077,0.093144,0.017719,8.060110,-0.141320,1.869989,0.203204,0.116418,0.111846,0.055273,0.016795,4.767101,-0.285868,2.052612,0.198017,0.464286,0.279070
3,1.0,0.433110,0.452199,0.413612,0.507393,0.185827,5.407515,-0.178233,2.877838,0.796796,0.313250,0.303628,0.177598,0.045371,4.937358,0.197943,2.053678,0.537011,0.535714,0.720930
4,1.0,0.174397,0.143835,0.092262,0.235594,0.087553,5.872954,-0.145480,2.194423,0.580544,0.075283,0.073279,0.089768,0.020443,5.257301,0.322600,1.961818,0.305234,0.300000,0.381818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6243,1.0,0.057205,0.090157,0.062531,0.239037,0.076481,5.343982,-1.784184,7.359470,0.284327,0.074408,0.071059,0.079528,0.019761,4.965753,-0.275006,1.981149,0.231365,0.357143,0.458333
6244,0.0,0.245866,0.220982,0.203962,0.288935,0.076996,7.427805,0.411869,2.231169,0.715673,0.150726,0.144354,0.131286,0.026695,5.351872,0.132997,1.846508,0.468198,0.642857,0.541667
6245,1.0,0.066812,0.117459,0.072057,0.250071,0.097960,5.643485,-1.266806,5.160196,0.431100,0.092283,0.085632,0.049652,0.014417,4.818442,-0.154963,1.998672,0.316035,0.307692,0.403226
6246,0.0,0.054204,0.055954,0.054394,0.061773,0.012936,9.272239,-0.002854,1.549116,0.212042,0.025605,0.024435,0.028404,0.007053,5.096906,-0.086395,1.811665,0.095090,0.102564,0.161290


In [578]:
y = df_train[0]
x = df_train.drop(0, axis = 1)
x_test = df_test.drop(0, axis = 1)
y_test = df_test[0]

In [579]:
x

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.046318,0.141910,0.084182,0.383329,0.139563,4.503520,-1.408795,6.647475,0.617802,0.094052,0.082395,0.069777,0.023589,4.426887,-0.164270,2.083033,0.360066,0.321429,0.500000
1,0.073971,0.087852,0.071337,0.180243,0.054842,6.338891,-0.645344,2.393239,0.382198,0.047816,0.044660,0.071250,0.027742,3.443768,0.475093,2.737538,0.194479,0.678571,0.500000
2,0.037709,0.061516,0.027380,0.159906,0.060017,4.312509,-0.654140,3.289271,0.189828,0.071038,0.065286,0.057454,0.014160,5.029416,-0.134997,1.937705,0.206033,0.159091,0.168539
3,0.088211,0.083918,0.081087,0.092759,0.017558,8.095644,0.162758,1.868488,0.267116,0.100426,0.096071,0.095724,0.021325,5.226386,0.239747,1.932730,0.302636,0.272727,0.123596
4,0.063538,0.065820,0.062985,0.062872,0.018931,7.696024,-0.682489,2.553929,0.207056,0.053089,0.051664,0.046562,0.011208,5.010536,-0.032268,1.960073,0.163216,0.181818,0.134831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6976,0.075688,0.116231,0.079073,0.313965,0.100537,4.813671,-1.512756,6.660133,0.306430,0.123269,0.121593,0.182902,0.055990,4.209587,-0.565761,2.472689,0.322430,0.466667,0.478261
6977,0.258272,0.254036,0.220336,0.393153,0.119428,7.099124,0.605659,2.548129,0.693570,0.208532,0.193873,0.217791,0.054846,5.097935,0.273143,1.997188,0.531131,0.533333,0.521739
6978,0.106475,0.137536,0.064309,0.302599,0.120021,5.708663,-1.248314,5.236364,0.440847,0.122621,0.115695,0.073606,0.025475,4.181087,-0.464833,2.518841,0.371926,0.274510,0.389610
6979,0.041224,0.043721,0.041356,0.066269,0.015119,9.613998,0.202926,1.489863,0.146142,0.027363,0.025981,0.026985,0.006103,5.217626,-0.239576,1.902420,0.085627,0.117647,0.129870


In [580]:
y

0       1.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
6976    1.0
6977    0.0
6978    1.0
6979    0.0
6980    0.0
Name: 0, Length: 6981, dtype: float64

In [581]:
input_layer = Input(shape =(x.shape[1], ))
act_function = 'tanh'
# Building the Encoder network
encoded = Dense(128, activation =act_function,
                activity_regularizer = regularizers.l1(10e-5))(input_layer)
encoded = Dense(64, activation =act_function,
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(32, activation =act_function,
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(16, activation =act_function,
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(8, activation =act_function,
                activity_regularizer = regularizers.l1(10e-5))(encoded)

  
# Building the Decoder network
decoded = Dense(16, activation =act_function)(encoded)
decoded = Dense(32, activation =act_function)(decoded)
decoded = Dense(64, activation =act_function)(decoded)
decoded = Dense(128, activation =act_function)(decoded)

  

output_layer = Dense(x.shape[1], activation ='relu')(decoded)

In [582]:
X_scaled = MinMaxScaler().fit_transform(x)
X_normal_scaled = X_scaled[y == 0]
X_fraud_scaled = X_scaled[y == 1]
X_test_scaled = MinMaxScaler().fit_transform(x_test)
X_test_normal_scaled = X_test_scaled[y_test == 0]
X_test_fraud_scaled = X_test_scaled[y_test == 1]

In [583]:
X_test_normal_scaled.shape

(3492, 19)

In [584]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer ="adam", loss ="mse")
  
# Training the Auto-encoder network
autoencoder.fit(X_scaled, X_scaled, epochs = 35)

Epoch 1/35
219/219 [==============================] - 1s 1ms/step - loss: 0.0097
Epoch 2/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 3/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0021
Epoch 4/35
219/219 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 5/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 6/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 7/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 8/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 9/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 10/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 11/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 12/35
219/219 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 13/35
219/219 [====

In [585]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])
hidden_representation.add(autoencoder.layers[3])
hidden_representation.add(autoencoder.layers[4])

In [586]:
# Separating the points encoded by the Auto-encoder as normal and fraud
normal_hidden_rep = hidden_representation.predict(X_normal_scaled)
fraud_hidden_rep = hidden_representation.predict(X_fraud_scaled)

normal_hidden_rep_test = hidden_representation.predict(X_test_normal_scaled)
fraud_hidden_rep_test = hidden_representation.predict(X_test_fraud_scaled)
  
# Combining the encoded points into a single table 
encoded_X = np.append(normal_hidden_rep, fraud_hidden_rep, axis = 0)
y_normal = np.zeros(normal_hidden_rep.shape[0])
y_fraud = np.ones(fraud_hidden_rep.shape[0])
encoded_y = np.append(y_normal, y_fraud)
X_train_encoded = encoded_X
y_train_encoded = encoded_y
X_test_encoded = np.append(normal_hidden_rep_test, fraud_hidden_rep_test, axis = 0)
y_normal = np.zeros(normal_hidden_rep_test.shape[0])
y_fraud = np.ones(fraud_hidden_rep_test.shape[0])
encoded_y = np.append(y_normal, y_fraud)
y_test_encoded = encoded_y



87/87 [==============================] - 0s 576us/step


In [587]:
# Building the logistic regression model
lrclf = LogisticRegression()
lrclf.fit(X_train_encoded, y_train_encoded)
  
# Storing the predictions of the linear model
y_pred_lrclf = lrclf.predict(X_test_encoded)
  
# Evaluating the performance of the linear model
print('Accuracy : '+str(accuracy_score(y_test_encoded, y_pred_lrclf)))

Accuracy : 0.7152688860435339


In [588]:
# Building the SVM model
svmclf = SVC()
svmclf.fit(X_train_encoded, y_train_encoded)
  
# Storing the predictions of the non-linear model
y_pred_svmclf = svmclf.predict(X_test_encoded)
  
# Evaluating the performance of the non-linear model
print('Accuracy : '+str(accuracy_score(y_test_encoded, y_pred_svmclf)))

Accuracy : 0.8040973111395646


In [589]:
import pickle as pkl
pkl.dump(X_train_encoded, open('./enc_x_train.pkl', 'wb'))
pkl.dump(X_test_encoded, open('./enc_x_test.pkl', 'wb'))
pkl.dump(y_test_encoded, open('./enc_y_test.pkl', 'wb'))
pkl.dump(y_train_encoded, open('./enc_y_train.pkl', 'wb'))
